In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
import zipfile
import os

zip_path = '/content/drive/MyDrive/face.zip'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall("/content/")

In [18]:
import torchvision

# Data Variables | Dataset from Kaggle
train_data = "/content/face/DATA/train/"
test_data = "/content/face/DATA/testing/"
list_train = ['Acne', 'Rosacea', 'Normal']  # The category we want to train the Model with.

import torch
from torchvision.models import resnet18
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Перемещаем модель на GPU, если она доступна
model = torchvision.models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 3)
model = model.to(device).eval()  # Перевод модели на устройство и перевод в режим eval

# Извлечение слоя "avgpool" для извлечения признаков
avgpool_layer = model.avgpool

# Пример использования признаков
input_tensor = torch.randn(1, 3, 224, 224).to(device)
features = avgpool_layer(model.conv1(input_tensor))  # Применение avgpool_layer к выходу conv1
print((features.squeeze(2,3)).shape)  # Размерность признаков

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


torch.Size([1, 64])


In [19]:
import os, cv2, numpy as np, pandas as pd

def data_gen(): # This module will scour through defined paths to find images & add them to our dictionary.
    k = 0
    # Dictionaries to store training & testing image paths and targets
    train_dictionary = {"img_path": [], "target": []}
    test_dictionary = {"img_path": [], "target": []}

    for i in list_train:
        path_disease_train = train_data + i
        path_disease_test = test_data + i

        image_list_train = os.listdir(path_disease_train)
        image_list_test = os.listdir(path_disease_test)

        for j in image_list_train:
            img_path_train = path_disease_train + "/" + j
            train_dictionary["img_path"].append(img_path_train)
            train_dictionary['target'].append(k)

        for m in image_list_test:
            img_path_test = path_disease_test + "/" + m
            test_dictionary["img_path"].append(img_path_test)
            test_dictionary['target'].append(k)
        k += 1

    # Create a testing & training DataFrame from the test & train dictionary.
    test_df = pd.DataFrame(test_dictionary)
    train_df = pd.DataFrame(train_dictionary)
    return train_df, test_df

In [20]:
from tensorflow.keras.applications.imagenet_utils import preprocess_input
from tensorflow.keras.models import Sequential, load_model
import matplotlib.pyplot as plt, random
from PIL import Image as PILImageHandler
from tensorflow.keras.layers import Dense

def load_data(input_size=(224, 224)):  # Function to load and preprocess the data
    images = []
    images2 = []
    train_df, test_df = data_gen()

    for i in train_df['img_path']:
        img = cv2.imread(i)
        img = cv2.resize(img, input_size)
        images.append(img)
    y_train = np.asarray(train_df['target'])
    x_train = np.asarray(images)

    for i in test_df['img_path']:
        img = cv2.imread(i)
        img = cv2.resize(img, input_size)
        images2.append(img)
    y_test = np.asarray(test_df['target'])
    x_test = np.asarray(images2)

    return x_train, x_test, y_train, y_test  # Return the preprocessed data

x_train, x_test, y_train, y_test = load_data(input_size=(224, 224))  # Load and preprocess the data

In [21]:
from PIL import Image
import torchvision.transforms as transforms

def load_img(img_path):
    # Загрузка изображения и предобработка
    img = Image.open(img_path).convert('RGB')
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    img = transform(img)
    img = torch.unsqueeze(img, 0)  # Добавление размерности пакета (batch dimension)

    # Перевод изображения на устройство
    img = img.to(device)

    # Применение модели и извлечение признаков
    with torch.no_grad():
        features = model(img)

    # Извлечение признаков и преобразование в массив NumPy
    features = features.squeeze().cpu().numpy()

    return features

img_path = "/content/face/DATA/testing/Acne/12.jpg"
img_features = load_img(img_path)
print(img_features.shape)

(3,)


In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms


# Определение класса для загрузки и предобработки данных
class SkinDataset(Dataset):
    def __init__(self, x, y, transform=None):
        self.x = x
        self.y = y
        self.transform = transform

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        img = self.x[idx]
        label = self.y[idx]
        if self.transform:
            img = self.transform(img)
        return img, label

# Предобработка данных
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = SkinDataset(x_train, y_train, transform=transform)
test_dataset = SkinDataset(x_test, y_test, transform=transform)

# Загрузка данных с помощью DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

# Определение функции потерь и оптимизатора
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Инициализация переменной для хранения наилучшей точности и соответствующих весов модели
best_accuracy = 0.0
best_model_weights = None

# Цикл обучения модели с выводом потери на каждой итерации
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader, 1):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        # Вывод потери на каждой итерации
        if i % 100 == 0:  # выведите потерю каждые 100 итераций
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i}/{len(train_loader)}], Loss: {loss.item():.4f}")

    # Подсчет и вывод средней потери на эпохе
    epoch_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

    # Оценка модели на тестовом наборе данных и сохранение модели с лучшими весами
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        # Рассчет точности на тестовом наборе данных
        accuracy = correct / total
        print(f"Accuracy on test set: {100 * accuracy:.2f}%")

        # Сохранение модели с наилучшими весами
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_model_weights = model.state_dict()

# Сохранение модели с наилучшими весами
torch.save(best_model_weights, 'best_model.pth')
print("Best model saved successfully!")

Epoch [1/20], Loss: 0.4890
Accuracy on test set: 56.03%
Epoch [2/20], Loss: 0.2028
Accuracy on test set: 67.24%
Epoch [3/20], Loss: 0.0980
Accuracy on test set: 81.03%
Epoch [4/20], Loss: 0.1397
Accuracy on test set: 95.69%
Epoch [5/20], Loss: 0.0879
Accuracy on test set: 87.93%
Epoch [6/20], Loss: 0.1004
Accuracy on test set: 85.34%
Epoch [7/20], Loss: 0.0578
Accuracy on test set: 95.69%
Epoch [8/20], Loss: 0.0351
Accuracy on test set: 88.79%
Epoch [9/20], Loss: 0.0201
Accuracy on test set: 94.83%
Epoch [10/20], Loss: 0.0108
Accuracy on test set: 85.34%
Epoch [11/20], Loss: 0.0158
Accuracy on test set: 90.52%
Epoch [12/20], Loss: 0.0317
Accuracy on test set: 56.90%
Epoch [13/20], Loss: 0.0921
Accuracy on test set: 81.03%
Epoch [14/20], Loss: 0.1943
Accuracy on test set: 80.17%
Epoch [15/20], Loss: 0.0844
Accuracy on test set: 87.07%
Epoch [16/20], Loss: 0.0599
Accuracy on test set: 87.93%
Epoch [17/20], Loss: 0.0621
Accuracy on test set: 67.24%
Epoch [18/20], Loss: 0.0310
Accuracy on 

In [23]:
# Сохранение весов наилучшей модели
torch.save(best_model_weights, 'best_model_weights.pth')
print("Best model weights saved successfully!")

Best model weights saved successfully!


In [27]:
model.load_state_dict(torch.load('/content/best_model_weights.pth'))
model.eval()

# Функция для загрузки и предобработки изображения лица
def preprocess_image(image_path):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    image = Image.open(image_path).convert('RGB')
    image = transform(image)
    image = torch.unsqueeze(image, 0)
    return image

# Путь к изображению лица
face_image_path = "/content/1.jpg"

# Предобработка изображения
face_image = preprocess_image(face_image_path)
face_image = preprocess_image(face_image_path).to(device)


# Передача изображения через модель для получения предсказаний
with torch.no_grad():
    outputs = model(face_image)

# Получение предсказанных вероятностей для каждого класса
probabilities = torch.softmax(outputs, dim=1)[0]
class_names = ['Acne', 'Rosacea', 'Normal']  # Предположим, что это порядок классов

# Вывод предсказаний
for i, prob in enumerate(probabilities):
    class_name = class_names[i]
    print(f"Probability of {class_name}: {prob.item()}")

# Пример анализа предсказаний и отображения результатов
threshold = 0.5  # Порог вероятности для классификации
if probabilities[0] > threshold:
    print("Acne detected!")
    # Дополнительные действия для обработки обнаружения акне на изображении
else:
    print("No acne detected.")

Probability of Acne: 0.0006646826514042914
Probability of Rosacea: 0.13390961289405823
Probability of Normal: 0.8654256463050842
No acne detected.


In [ ]:
userInput = str(input("Which Skin disease to compare with our model?\n Acne, Rosacea or Eczema?: "))
img_number = str(input("Will you enter the number of the image or should it be random?\n Filename: "))

base_path = "face_data/DATA/train/"+userInput+"/"

if img_number.lower() == "random":
    files = os.listdir(base_path)
    file = [random.choice(files)]
    final = base_path + f"{file[0]}"; print(final)
    print(f"The random file chosen is: {file}\n The final path is: {final}")

else:
    final = base_path + img_number + ".jpg"
    print(f"The file chosen is: {img_number}\n The final path is: {final}")

img_path = final
# model.save('saved_model/skin_model')
# model = load_model('saved_model/skin_model')
img = load_img(img_path)
np.argmax(model.predict(img))  # Make a prediction using the trained model.

img = np.expand_dims(cv2.resize(cv2.imread(img_path), (100, 100)), axis=0)
img_features = VGG.predict(preprocess_input(img)).reshape(1, 4608)
prediction = np.argmax(model.predict(img_features))

# Convert the predicted class back to its original label
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
predicted_label = label_encoder.inverse_transform([prediction])[0]
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

#predicted_label = np.argmax(prediction)
predicted_class = list_train[predicted_label]

# Print the predicted class

def openImage():
    im = PILImageHandler.open(f"{img_path}")
    im.show()

root = Tk()
frm = ttk.Frame(root, padding=20)
frm.grid()
ttk.Label(frm, text=f"The predicted class for {img_path} is {list_train[predicted_label]}").grid(column=0, row=0)
ttk.Button(frm, text="See image", command=openImage).grid(column=1, row=0)
root.mainloop()

print(f"The predicted class for {img_path} is {list_train[predicted_label]}")